In [ ]:

####################################################################################################
####################################################################################################

import sys
import os.path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display

import keras
import keras.utils.vis_utils
from keras import backend as K
from keras import layers
from keras.layers import Dense
from keras.models import load_model
from keras.models import Sequential
from keras.utils import plot_model

sys.path.append(os.path.abspath("../"))
from helpers import *

sys.path.append(os.path.abspath("../../"))
from pax_utils import s1s2_utils

v_drift = 1.44e-4


Using TensorFlow backend.


In [ ]:

####################################################################################################
####################################################################################################

nEventsTrain = 100000
file_all     = "../data_lfs/merged_all_200k_1803261941.pkl"

df_all = pd.read_pickle(file_all)
df_all = df_all[df_all['intr_count'] == 1].reset_index(drop=True)
df_all = df_all[nEventsTrain:][:]

df_all['intr_depth'] = -1*v_drift*df_all['intr_drift_time'] 
df_all['depth']      *= -1


####################################################################################################
####################################################################################################

df_train_truth = df_all[:][['x', 'y', 'depth', 's2_electrons']]
df_train_input = df_all[:][['intr_depth'] + s1s2_utils.getS2integralsDataFrameColumns()]


####################################################################################################
####################################################################################################

checkTrainingData(df_train_input, df_train_truth)



In [ ]:

####################################################################################################
####################################################################################################

model_name     = "model_barge_xyzS2_elu"
folder         = "models" + "/"    
model_name_h5  = folder + model_name + ".h5"
model_name_png = folder + model_name + ".png"



In [ ]:

####################################################################################################
####################################################################################################

model       = load_model(model_name_h5)
arr_xy_pred = model.predict(df_train_input)

print("\nLoaded Model: " + model_name_h5)
print("Predicted\n")



In [ ]:

####################################################################################################
####################################################################################################

arr_x_true = df_train_truth[:]['x'].as_matrix()
arr_y_true = df_train_truth[:]['y'].as_matrix()
arr_z_true = df_train_truth[:]['depth'].as_matrix()

arr_s2_true = df_train_truth[:]['s2_electrons'].as_matrix()
arr_s2_reco = df_all[:]['intr_s2_electrons'].as_matrix()

arr_x_pred  = arr_xy_pred[:, 0]
arr_y_pred  = arr_xy_pred[:, 1]
arr_z_pred  = arr_xy_pred[:, 2]
arr_s2_pred = arr_xy_pred[:, 3]



In [ ]:

####################################################################################################
####################################################################################################

file_out         = "./predictions/xyzS2_S2integralsDeltaT.pkl"

cols_out = ['event_number',
            'x', 'y', 'depth',
            'intr_x', 'intr_y', 'intr_z',
            'intr_x_nn', 'intr_y_nn',
            's2_electrons', 'intr_s2_electrons']

df_out                      = df_all[:][cols_out]
df_out['x_pred']            = arr_x_pred
df_out['y_pred']            = arr_y_pred
df_out['z_pred']            = arr_z_pred
df_out['s2_electrons_pred'] = arr_s2_pred


df_out.to_pickle(file_out)


####################################################################################################
####################################################################################################

df_test = pd.read_pickle(file_out)

display(df_test[0:5][:])
print()

